# 🐦 Zero-Shot BirdDetector — Initial Exploration

### 1. Setup

In [ ]:
import torch
from deepforest import DeepForest
from deepforest import visualize
import matplotlib.pyplot as plt
import numpy as np
import cv2

### 2. Load the Model

In [ ]:
model = DeepForest()
model.use_bird_release() # BirdDetector model

### 3. Load a Sample Drone Image

In [ ]:
image = cv2.imread("path/to/test.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)
plt.axis("off")

### 4. Run Prediction

In [ ]:
predictions = model.predict_image(image=image)
predictions

### 5. Visualize Predictions

In [ ]:
visualize.plot_predictions(image, predictions)

### 6. Evaluate (Basic)

If dataset has boxes or points.

### 7. Notes + Observations

You will write:
- How well zero-shot works
- What birds it misses
- Where false positives happen
- Habitat patterns
